In [1]:
import pandas as pd
import glob
import os

data_path = "Data"

# Assistant
# Find all parquet files matching the pattern for 2024 yellow taxi data and sort them
files = sorted(glob.glob(os.path.join(data_path, "yellow_tripdata_2024-*.parquet")))

# Read the first file to establish the column structure
# This ensures consistent columns across all files
df_first = pd.read_parquet(files[0])
columns = df_first.columns

# Store the first dataframe in our list of dataframes to concatenate
dataframes = [df_first]

# Process each remaining file, ensuring column consistency
for file in files[1:]:
    df = pd.read_parquet(file)
    
    # Align columns: add missing columns with NaN values, drop unexpected columns
    # This ensures all dataframes have identical structure before concatenation
    df = df.reindex(columns=columns)
    
    dataframes.append(df)

# Concatenate all monthly dataframes into one large dataframe
# ignore_index=True creates a new sequential index rather than keeping original indices
df_all = pd.concat(dataframes, ignore_index=True)

# Define a mapping dictionary to rename columns to more descriptive and consistent names
# This improves readability and follows better naming conventions
column_rename_map = {
    "VendorID": "data_vendor_id",
    "tpep_pickup_datetime": "trip_start_datetime",
    "tpep_dropoff_datetime": "trip_end_datetime",
    "passenger_count": "number_of_passengers",
    "trip_distance": "trip_distance_miles",
    "RatecodeID": "fare_rate_code_id",
    "store_and_fwd_flag": "stored_and_forwarded_flag",
    "PULocationID": "pickup_location_id",
    "DOLocationID": "dropoff_location_id",
    "payment_type": "payment_method_code",
    "fare_amount": "base_fare_amount",
    "extra": "additional_surcharges_amount",
    "mta_tax": "mta_tax_amount",
    "tip_amount": "tip_amount",
    "tolls_amount": "tolls_amount",
    "improvement_surcharge": "improvement_surcharge_amount",
    "congestion_surcharge": "congestion_surcharge_amount",
    "Airport_fee": "airport_fee_amount",
    "total_amount": "total_trip_amount"
}

# Apply the column renaming to the dataframe
df_all = df_all.rename(columns=column_rename_map)

# Convert specific columns to nullable integer types (Int64)
# This handles potential NaN values better than standard int types
df_all["number_of_passengers"] = df_all["number_of_passengers"].astype("Int64")
df_all["fare_rate_code_id"] = df_all["fare_rate_code_id"].astype("Int64")
df_all["payment_method_code"] = df_all["payment_method_code"].astype("Int64")

print("Combined shape:", df_all.shape)
df_all.head()

Combined shape: (41169720, 19)


,data_vendor_id,trip_start_datetime,trip_end_datetime,number_of_passengers,trip_distance_miles,fare_rate_code_id,stored_and_forwarded_flag,pickup_location_id,dropoff_location_id,payment_method_code,base_fare_amount,additional_surcharges_amount,mta_tax_amount,tip_amount,tolls_amount,improvement_surcharge_amount,total_trip_amount,congestion_surcharge_amount,airport_fee_amount
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.80,1,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1,4.70,1,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1,1.40,1,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1,0.80,1,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [2]:
# Assistant
# Create a random sample of 1000 rows from the full dataset
# Using random_state=42 ensures reproducibility of the sample
df_sample = df_all.sample(n=1000, random_state=42)

# Export the sampled data to a CSV file in the specified data directory
# index=False prevents writing row indices to the output file
df_sample.to_csv(os.path.join(data_path, "yellow_tripdata_sample_1000.csv"), index=False)

# Print confirmation message and display the dimensions of the sample dataset
print("Sample saved. Shape:", df_sample.shape)

Sample saved. Shape: (1000, 19)


In [3]:
# Assistant
# Define the column to analyze
col = "data_vendor_id"

# Print basic information about the column
print("Total rows:", len(df_all))
print("Null rows:", df_all[col].isna().sum())
print("Data type:", df_all[col].dtype, "\n")

# Display value counts including null values
print(df_all[col].value_counts(dropna=False), "\n")

# Define the set of expected/valid vendor IDs
EXPECTED_VENDOR_IDS = {1, 2}

# Create a boolean mask to identify invalid vendor IDs
# (rows that are not null AND not in our expected set)
invalid_mask = (
    df_all[col].notna()
    & ~df_all[col].isin(EXPECTED_VENDOR_IDS)
)

# Display the counts of each invalid vendor ID
print(df_all.loc[invalid_mask, col].value_counts(), "\n")

# Calculate statistics about invalid rows
invalid_count = invalid_mask.sum()
total_count = len(df_all)

# Print the number and percentage of invalid rows
print(
    f"Invalid rows: {invalid_count} "
    f"({invalid_count / total_count:.4%} of dataset)", "\n"
)

# Remove rows with invalid vendor IDs from the dataframe
invalid_idx = df_all.index[invalid_mask]
df_all.drop(index=invalid_idx, inplace=True)

# Verify the cleaning by displaying the updated value counts
print("Vendor ID-s after cleaning:", '\n')
print(df_all[col].value_counts(dropna=False), "\n")

Total rows: 41169720
Null rows: 0
Data type: int32 

data_vendor_id
2    31451503
1     9715918
6        2069
7         230
 ame: count, dtype: int64

data_vendor_id
6    2069
7     230
Name: count, dtype: int64 

Invalid rows: 2299 (0.0056% of dataset) 

Vendor ID-s after cleaning: 

data_vendor_id
2    31451503
1     9715918
Name: count, dtype: int64 



In [4]:
# Assistant
col = "trip_start_datetime"

# Basic diagnostics
print("Total rows:", len(df_all))
print("Null rows:", df_all[col].isna().sum())
print("Data type:", df_all[col].dtype, "\n")

# Display statistical summary of the datetime column
print("Value distribution summary:")
print(df_all[col].describe(), "\n")

# Define valid date range boundaries for trip start times
# - We expect all valid trips to start in 2024
# - Any trips outside this range are considered invalid
MIN_DATE = pd.Timestamp("2024-01-01 00:00:00")
MAX_DATE = pd.Timestamp("2025-01-01 00:00:00")

# Helper function to identify invalid datetime values
# Returns a boolean mask where True indicates an invalid value
def get_invalid_mask(df, column, min_value, max_value):
    return df[column].notna() & ((df[column] < min_value) | (df[column] >= max_value))

# Identify rows with invalid trip start times
invalid_mask = get_invalid_mask(df_all, col, MIN_DATE, MAX_DATE)
print("Rows with invalid values before cleaning:", invalid_mask.sum())

# Remove rows with invalid trip start times
invalid_idx = df_all.index[invalid_mask]
df_all.drop(index=invalid_idx, inplace=True)

# Verify that all invalid values have been removed
invalid_mask = get_invalid_mask(df_all, col, MIN_DATE, MAX_DATE)
print("Rows with invalid values after cleaning:", invalid_mask.sum())

# Final data quality check after cleaning
print("Null rows after cleaning:", df_all[col].isna().sum())
print("Value distribution summary after cleaning:")
print(df_all[col].describe(), "\n")

Total rows: 41167421
Null rows: 0
Data type: datetime64[us] 

Value distribution summary:
count                      41167421
mean     2024-07-06 10:00:42.755670
min             2002-12-31 16:46:07
25%             2024-04-06 20:09:50
50%             2024-07-03 23:37:02
75%             2024-10-08 17:31:40
max             2026-06-26 23:53:12
Name: trip_start_datetime, dtype: object 

Rows with invalid values before cleaning: 56
Rows with invalid values after cleaning: 0
Null rows after cleaning: 0
Value distribution summary after cleaning:
count                      41167365
mean     2024-07-06 10:09:15.688188
min             2024-01-01 00:00:00
25%             2024-04-06 20:10:05
50%             2024-07-03 23:37:37
75%             2024-10-08 17:31:44
max             2024-12-31 23:59:58
Name: trip_start_datetime, dtype: object 



In [5]:
# Assistant
col = "trip_end_datetime"

# Basic diagnostics
print("Total rows:", len(df_all))
print("Null rows:", df_all[col].isna().sum())
print("Data type:", df_all[col].dtype, "\n")

# Display statistical summary of the datetime column
print("Value distribution summary:")
print(df_all[col].describe(), "\n")

# Define valid date range boundaries for trip start times
# - We expect all valid trips to start in 2024
# - Any trips outside this range are considered invalid
MIN_DATE = pd.Timestamp("2024-01-01 00:00:00")
MAX_DATE = pd.Timestamp("2025-01-01 00:00:00")

# Helper function to identify invalid datetime values
# Returns a boolean mask where True indicates an invalid value
def get_invalid_mask(df, column, min_value, max_value):
    return df[column].notna() & ((df[column] < min_value) | (df[column] >= max_value))

# Identify rows with invalid trip start times
invalid_mask = get_invalid_mask(df_all, col, MIN_DATE, MAX_DATE)
print("Rows with invalid values before cleaning:", invalid_mask.sum())

# Remove rows with invalid trip start times
invalid_idx = df_all.index[invalid_mask]
df_all.drop(index=invalid_idx, inplace=True)

# Verify that all invalid values have been removed
invalid_mask = get_invalid_mask(df_all, col, MIN_DATE, MAX_DATE)
print("Rows with invalid values after cleaning:", invalid_mask.sum())

# Final data quality check after cleaning
print("Null rows after cleaning:", df_all[col].isna().sum())
print("Value distribution summary after cleaning:")
print(df_all[col].describe(), "\n")

Total rows: 41167365
Null rows: 0
Data type: datetime64[us] 

Value distribution summary:
count                      41167365
mean     2024-07-06 10:26:43.677426
min             2024-01-01 00:02:42
25%             2024-04-06 20:26:12
50%             2024-07-03 23:53:19
75%             2024-10-08 17:51:46
max             2025-01-01 22:59:33
Name: trip_end_datetime, dtype: object 

Rows with invalid values before cleaning: 611
Rows with invalid values after cleaning: 0
Null rows after cleaning: 0
Value distribution summary after cleaning:
count                      41166754
mean     2024-07-06 10:22:54.632399
min             2024-01-01 00:02:42
25%             2024-04-06 20:25:04
50%      2024-07-03 23:46:13.500000
75%             2024-10-08 17:48:35
max             2024-12-31 23:59:59
Name: trip_end_datetime, dtype: object 



In [6]:
# Assistant
# Define the column to analyze
col = "number_of_passengers"

# Display basic information about the column
print("Total rows:", len(df_all))
print("Null rows:", df_all[col].isna().sum())
print("Data type:", df_all[col].dtype, "\n")

# Show the distribution of values including null values
print("Value distribution (including nulls):")
print(df_all[col].value_counts(dropna=False).sort_index(), "\n")

# Create a mask to identify rows where passenger count is zero
zero_mask = df_all[col] == 0
print("Rows with zero passengers before cleaning:", zero_mask.sum(), '\n')

# Replace zero passenger counts with NA values (data cleaning)
df_all.loc[zero_mask, col] = pd.NA

# Verify that zero values were successfully replaced
zero_mask = df_all[col] == 0
print("Rows with zero passengers after cleaning:", zero_mask.sum(), '\n')

# Display the updated value distribution after cleaning
print("Value distribution after cleaning (including nulls):")
print(df_all[col].value_counts(dropna=False).sort_index())

Total rows: 41166754
Null rows: 4089128
Data type: Int64 

Value distribution (including nulls):
number_of_passengers
0         401351
1       28632114
2        5410593
3        1282030
4         814855
5         320605
6         215794
7             56
8            192
9             36
<NA>     4089128
Name: count, dtype: Int64 

Rows with zero passengers before cleaning: 401351 

Rows with zero passengers after cleaning: 0 

Value distribution after cleaning (including nulls):
number_of_passengers
1       28632114
2        5410593
3        1282030
4         814855
5         320605
6         215794
7             56
8            192
9             36
<NA>     4490479
Name: count, dtype: Int64


In [7]:
col = "fare_rate_code_id"

print("Total rows:", len(df_all))
print("Null rows:", df_all[col].isna().sum())
print("Data type:", df_all[col].dtype, "\n")

print("Value distribution (including nulls):")
print(df_all[col].value_counts(dropna=False).sort_index(), "\n")

# Valid TLC Rate Code IDs (Yellow Taxi standard):
# 1 = Standard rate
# 2 = JFK
# 3 = Newark
# 4 = Nassau or Westchester
# 5 = Negotiated fare
# 6 = Group ride
EXPECTED_RATE_CODE_IDS = {1, 2, 3, 4, 5, 6}

def get_invalid_mask(df, column, expected_values):
    return df[column].notna() & ~df[column].isin(expected_values)

invalid_mask = get_invalid_mask(df_all, col, EXPECTED_RATE_CODE_IDS)
print("Rows with invalid codes before cleaning:", invalid_mask.sum(), '\n')

df_all.loc[invalid_mask, col] = pd.NA

invalid_mask = get_invalid_mask(df_all, col, EXPECTED_RATE_CODE_IDS)
print("Rows with invalid codes after cleaning:", invalid_mask.sum(), '\n')

print("Value distribution after cleaning (including nulls):")
print(df_all[col].value_counts(dropna=False).sort_index())

Total rows: 41166754
Null rows: 4089128
Data type: Int64 

Value distribution (including nulls):
fare_rate_code_id
1       34650263
2        1406796
3         129950
4         101630
5         321937
6             76
99        466974
<NA>     4089128
 ame: count, dtype: Int64

Rows with invalid codes before cleaning: 466974 

Rows with invalid codes after cleaning: 0 

Value distribution after cleaning (including nulls):
fare_rate_code_id
1       34650263
2        1406796
3         129950
4         101630
5         321937
6             76
<NA>     4556102
Name: count, dtype: Int64


In [8]:
# Assistant
col = "fare_rate_code_id"

# Display basic information about the column
print("Total rows:", len(df_all))
print("Null rows:", df_all[col].isna().sum())
print("Data type:", df_all[col].dtype, "\n")

# Show the distribution of values in the column, including null values
print("Value distribution (including nulls):")
print(df_all[col].value_counts(dropna=False).sort_index(), "\n")

# Define the valid rate code IDs according to TLC Yellow Taxi standards
# 1 = Standard rate
# 2 = JFK
# 3 = Newark
# 4 = Nassau or Westchester
# 5 = Negotiated fare
# 6 = Group ride
EXPECTED_RATE_CODE_IDS = {1, 2, 3, 4, 5, 6}

# Helper function to identify rows with invalid values
# Returns a boolean mask where True indicates an invalid value
def get_invalid_mask(df, column, expected_values):
    return df[column].notna() & ~df[column].isin(expected_values)

# Identify rows with invalid rate codes
invalid_mask = get_invalid_mask(df_all, col, EXPECTED_RATE_CODE_IDS)
print("Rows with invalid codes before cleaning:", invalid_mask.sum(), '\n')

# Replace invalid rate codes with NA values
df_all.loc[invalid_mask, col] = pd.NA

# Verify that all invalid values have been replaced
invalid_mask = get_invalid_mask(df_all, col, EXPECTED_RATE_CODE_IDS)
print("Rows with invalid codes after cleaning:", invalid_mask.sum(), '\n')

# Display the updated distribution of values after cleaning
print("Value distribution after cleaning (including nulls):")
print(df_all[col].value_counts(dropna=False).sort_index())

Total rows: 41166754
Null rows: 4556102
Data type: Int64 

Value distribution (including nulls):
fare_rate_code_id
1       34650263
2        1406796
3         129950
4         101630
5         321937
6             76
<NA>     4556102
 ame: count, dtype: Int64

Rows with invalid codes before cleaning: 0 

Rows with invalid codes after cleaning: 0 

Value distribution after cleaning (including nulls):
fare_rate_code_id
1       34650263
2        1406796
3         129950
4         101630
5         321937
6             76
<NA>     4556102
Name: count, dtype: Int64


In [9]:
# Assistant
col = "stored_and_forwarded_flag"

# Display basic information about the column
print("Total rows:", len(df_all))
print("Null rows:", df_all[col].isna().sum())
print("Data type:", df_all[col].dtype, "\n")

# Show the distribution of values in the column, including null values
print("Value distribution (including nulls):")
print(df_all[col].value_counts(dropna=False), "\n")

# No cleaning needed as the column only contains expected values and nulls

Total rows: 41166754
Null rows: 4089128
Data type: object 

Value distribution (including nulls):
stored_and_forwarded_flag
N       36902456
None     4089128
Y         175170
Name: count, dtype: int64 



In [10]:
# Assistant
# Define the column to analyze
col = "pickup_location_id"

# Display basic information about the column
print("Total rows:", len(df_all))
print("Null rows:", df_all[col].isna().sum())
print("Data type:", df_all[col].dtype, "\n")

# Show the distribution of values in the column (including null values)
print("Value distribution (including nulls):")
print(df_all[col].value_counts(dropna=False).sort_index(), "\n")

# Define the expected range of location IDs (1 through 263)
EXPECTED_LOCATION_IDS = set(range(1, 264))

# Helper function to identify rows with invalid values
def get_invalid_mask(df, column, expected_values):
    # Returns a boolean mask where True indicates values that exist but are not in expected_values
    return df[column].notna() & ~df[column].isin(expected_values)

# Apply the function to identify invalid location IDs
invalid_mask = get_invalid_mask(df_all, col, EXPECTED_LOCATION_IDS)
print("Rows with invalid codes before cleaning:", invalid_mask.sum())

# Get indices of invalid rows and remove them from the dataframe
invalid_idx = df_all.index[invalid_mask]
df_all.drop(index=invalid_idx, inplace=True)

# Verify that all invalid values have been removed
invalid_mask = get_invalid_mask(df_all, col, EXPECTED_LOCATION_IDS)
print("Rows with invalid codes after cleaning:", invalid_mask.sum())

# Display the updated distribution of values after cleaning
print("Value distribution after cleaning (including nulls):")
print(df_all[col].value_counts(dropna=False).sort_index())

Total rows: 41166754
Null rows: 0
Data type: int32 

Value distribution (including nulls):
pickup_location_id
1        5940
2          62
3        1725
4       71457
5           7
        ...  
261    218052
262    562784
263    766730
264    122197
265     22751
Name: count, Length: 263, dtype: int64 

Rows with invalid codes before cleaning: 144948
Rows with invalid codes after cleaning: 0
Value distribution after cleaning (including nulls):
pickup_location_id
1        5940
2          62
3        1725
4       71457
5           7
        ...  
259      2222
260     13154
261    218052
262    562784
263    766730
Name: count, Length: 261, dtype: int64


In [11]:
# Assistant
# Define the column to analyze
col = "dropoff_location_id"

# Display basic information about the column
print("Total rows:", len(df_all))
print("Null rows:", df_all[col].isna().sum())
print("Data type:", df_all[col].dtype, "\n")

# Show the distribution of values in the column (including null values)
print("Value distribution (including nulls):")
print(df_all[col].value_counts(dropna=False).sort_index(), "\n")

# Define the expected range of location IDs (1 through 263)
EXPECTED_LOCATION_IDS = set(range(1, 264))

# Helper function to identify rows with invalid values
def get_invalid_mask(df, column, expected_values):
    # Returns a boolean mask where True indicates values that exist but are not in expected_values
    return df[column].notna() & ~df[column].isin(expected_values)

# Apply the function to identify invalid location IDs
invalid_mask = get_invalid_mask(df_all, col, EXPECTED_LOCATION_IDS)
print("Rows with invalid codes before cleaning:", invalid_mask.sum())

# Get indices of invalid rows and remove them from the dataframe
invalid_idx = df_all.index[invalid_mask]
df_all.drop(index=invalid_idx, inplace=True)

# Verify that all invalid values have been removed
invalid_mask = get_invalid_mask(df_all, col, EXPECTED_LOCATION_IDS)
print("Rows with invalid codes after cleaning:", invalid_mask.sum())

# Display the updated distribution of values after cleaning
print("Value distribution after cleaning (including nulls):")
print(df_all[col].value_counts(dropna=False).sort_index())

Total rows: 41021806
Null rows: 0
Data type: int32 

Value distribution (including nulls):
dropoff_location_id
1      110792
2          81
3        3643
4      164351
5         182
        ...  
261    209652
262    612857
263    832758
264    114820
265    164609
Name: count, Length: 262, dtype: int64 

Rows with invalid codes before cleaning: 279429
Rows with invalid codes after cleaning: 0
Value distribution after cleaning (including nulls):
dropoff_location_id
1      110792
2          81
3        3643
4      164351
5         182
        ...  
259      4999
260     29378
261    209652
262    612857
263    832758
Name: count, Length: 260, dtype: int64


In [12]:
# Assistant
# Define the column to analyze
col = "payment_method_code"

# Display basic information about the column
print("Total rows:", len(df_all))
print("Null rows:", df_all[col].isna().sum())
print("Data type:", df_all[col].dtype, "\n")

# Show the distribution of values in the column, including null values
print("Value distribution (including nulls):")
print(df_all[col].value_counts(dropna=False).sort_index(), "\n")

# Valid TLC payment type codes:
# 1 = Credit card
# 2 = Cash
# 3 = No charge
# 4 = Dispute
# 5 = Unknown
# 6 = Voided trip
EXPECTED_PAYMENT_CODES = {1, 2, 3, 4, 5, 6}

# Define a function to identify rows with invalid values
def get_invalid_mask(df, column, expected_values):
    """
    Creates a boolean mask identifying rows with values that are not null
    and not in the expected values set.
    """
    return df[column].notna() & ~df[column].isin(expected_values)

# Identify rows with invalid payment codes
invalid_mask = get_invalid_mask(df_all, col, EXPECTED_PAYMENT_CODES)
print("Rows with invalid codes before cleaning:", invalid_mask.sum())

# Replace invalid payment codes with NA
df_all.loc[invalid_mask, col] = pd.NA

# Verify that all invalid codes have been removed
invalid_mask = get_invalid_mask(df_all, col, EXPECTED_PAYMENT_CODES)
print("Rows with invalid codes after cleaning:", invalid_mask.sum())

# Display the updated distribution of values
print("Value distribution after cleaning (including nulls):")
print(df_all[col].value_counts(dropna=False).sort_index())

Total rows: 40742377
Null rows: 0
Data type: Int64 

Value distribution (including nulls):
payment_method_code
0     4080646
1    30137475
2     5457626
3      283726
4      782904
Name: count, dtype: Int64 

Rows with invalid codes before cleaning: 4080646
Rows with invalid codes after cleaning: 0
Value distribution after cleaning (including nulls):
payment_method_code
1       30137475
2        5457626
3         283726
4         782904
<NA>     4080646
Name: count, dtype: Int64


In [13]:
# Assistant
# Define the column to analyze
col = "total_trip_amount"

# Print basic information about the column
print("Total rows:", len(df_all))
print("Null rows:", df_all[col].isna().sum())
print("Data type:", df_all[col].dtype, "\n")

# Display statistical summary of the column values
print("Value distribution summary:")
print(df_all[col].describe(), "\n")

# Define acceptable value range for the total trip amount
MIN_TOTAL_AMOUNT = 0  # Minimum acceptable value (no negative trip amounts)
MAX_REASONABLE_TOTAL = 1000  # Maximum reasonable value, used as a threshold for outliers

# Helper function to identify invalid values based on specified range
def get_invalid_mask(df, column, min_value, max_value):
    """
    Creates a boolean mask identifying values outside the specified range.
    """
    return df[column].notna() & ((df[column] < min_value) | (df[column] > max_value))

# Check how many invalid values exist before cleaning
invalid_mask = get_invalid_mask(df_all, col, MIN_TOTAL_AMOUNT, MAX_REASONABLE_TOTAL)
print("Rows with invalid values before cleaning:", invalid_mask.sum())

# Replace invalid values with NA
df_all.loc[invalid_mask, col] = pd.NA

# Verify that invalid values were successfully removed
invalid_mask = get_invalid_mask(df_all, col, MIN_TOTAL_AMOUNT, MAX_REASONABLE_TOTAL)
print("Rows with invalid values after cleaning:", invalid_mask.sum())

# Report the final count of null values after cleaning
print("Null rows after cleaning:", df_all[col].isna().sum())

Total rows: 40742377
Null rows: 0
Data type: float64 

Value distribution summary:
count    4.074238e+07
mean     2.742164e+01
std      7.749884e+01
min     -2.265450e+03
25%      1.572000e+01
50%      2.100000e+01
75%      3.024000e+01
max      3.355509e+05
Name: total_trip_amount, dtype: float64 

Rows with invalid values before cleaning: 600606
Rows with invalid values after cleaning: 0
Null rows after cleaning: 600606


In [14]:
# Assistant
# Define the column name we're analyzing
col = "trip_distance_miles"

# Print basic information about the column
print("Total rows:", len(df_all))
print("Null rows:", df_all[col].isna().sum())
print("Data type:", df_all[col].dtype, "\n")

# Display statistical summary of the column values
print("Value distribution summary:")
print(df_all[col].describe(), "\n")

# Define reasonable boundaries for trip distance
MIN_DISTANCE = 0  # Trips can't have negative distance
MAX_REASONABLE_DISTANCE = 50  # Set upper limit for reasonable trip distance in miles

# Helper function to identify invalid values based on min/max thresholds
def get_invalid_mask(df, column, min_value, max_value):
    return df[column].notna() & ((df[column] < min_value) | (df[column] > max_value))

# Identify rows with invalid distance values
invalid_mask = get_invalid_mask(df_all, col, MIN_DISTANCE, MAX_REASONABLE_DISTANCE)
print("Rows with invalid values before cleaning:", invalid_mask.sum())

# Get indices of invalid rows and remove them from the dataframe
invalid_idx = df_all.index[invalid_mask]
df_all.drop(index=invalid_idx, inplace=True)

# Verify that all invalid values have been removed
invalid_mask = get_invalid_mask(df_all, col, MIN_DISTANCE, MAX_REASONABLE_DISTANCE)
print("Rows with invalid values after cleaning:", invalid_mask.sum())

Total rows: 40742377
Null rows: 0
Data type: float64 

Value distribution summary:
count    4.074238e+07
mean     4.889713e+00
std      4.177569e+02
min      0.000000e+00
25%      1.010000e+00
50%      1.750000e+00
75%      3.310000e+00
max      3.986086e+05
Name: trip_distance_miles, dtype: float64 

Rows with invalid values before cleaning: 1461
Rows with invalid values after cleaning: 0


In [15]:
# Assistant
# Import necessary libraries
import numpy as np
import pandas as pd

# Convert string datetime columns to pandas datetime objects
# errors="coerce" will set invalid parsing as NaT (Not a Time)
df_all["trip_start_datetime"] = pd.to_datetime(df_all["trip_start_datetime"], errors="coerce")
df_all["trip_end_datetime"] = pd.to_datetime(df_all["trip_end_datetime"], errors="coerce")

# Calculate trip duration by subtracting start time from end time
# This creates a timedelta object
df_all["trip_duration"] = df_all["trip_end_datetime"] - df_all["trip_start_datetime"]

# Convert trip duration to seconds
df_all["trip_duration_seconds"] = df_all["trip_duration"].dt.total_seconds()
# Convert seconds to minutes and round to 2 decimal places
df_all["trip_duration_minutes"] = (df_all["trip_duration_seconds"] / 60.0).round(2)

# Display statistical summary of the trip duration columns
print(df_all[["trip_duration_seconds", "trip_duration_minutes"]].describe())

       trip_duration_seconds  trip_duration_minutes
count           4.074092e+07           4.074092e+07
mean            1.041749e+03           1.736248e+01
std             2.036042e+03           3.393403e+01
min            -8.562300e+04          -1.427050e+03
25%             4.690000e+02           7.820000e+00
50%             7.770000e+02           1.295000e+01
75%             1.260000e+03           2.100000e+01
max             5.860510e+05           9.767520e+03


In [16]:
# Assistant
# Define the column name for trip duration
col = 'trip_duration_seconds'

# Set minimum and maximum acceptable values for trip duration
MIN_DURATION_SECONDS = 30
MAX_REASONABLE_DURATION_SECONDS = 10800 # 3 hours

# Function to create a boolean mask identifying invalid values in a dataframe column
def get_invalid_mask(df, column, min_value, max_value):
    return df[column].notna() & ((df[column] < min_value) | (df[column] > max_value))

# Create a mask to identify rows with invalid trip durations
invalid_mask = get_invalid_mask(df_all, col, MIN_DURATION_SECONDS, MAX_REASONABLE_DURATION_SECONDS)
print("Rows with invalid values before cleaning:", invalid_mask.sum())

# Get indices of invalid rows and remove them from the dataframe
invalid_idx = df_all.index[invalid_mask]
df_all.drop(index=invalid_idx, inplace=True)

# Verify that all invalid values have been removed
invalid_mask = get_invalid_mask(df_all, col, MIN_DURATION_SECONDS, MAX_REASONABLE_DURATION_SECONDS)
print("Rows with invalid values after cleaning:", invalid_mask.sum())

Rows with invalid values before cleaning: 384397
Rows with invalid values after cleaning: 0


In [17]:
# Assistant
# Check how many duplicate rows exist in the dataframe
print("Number of duplicated rows", df_all.duplicated().sum())
# Remove all duplicate rows from the dataframe (modifying it in place)
df_all.drop_duplicates(inplace=True)
# Confirm to the user that duplicates have been removed
print("Duplicated rows deleted")

Number of duplicated rows 2
Duplicated rows deleted


In [18]:
# Assistant
output_columns = [
    # --- Identifiers / time ---
    "trip_start_datetime",
    "trip_end_datetime",
    "trip_duration_seconds",
    "trip_duration_minutes",

    # --- Cleaned categorical fields ---
    "data_vendor_id",
    "fare_rate_code_id",
    "stored_and_forwarded_flag",
    "payment_method_code",

    # --- Cleaned numeric fields ---
    "number_of_passengers",
    "trip_distance_miles",
    "total_trip_amount",

    # --- Location IDs (validated) ---
    "pickup_location_id",
    "dropoff_location_id",
]

# Create a new dataframe with only the selected columns
df_output = df_all[output_columns].copy()

# Display the dimensions of the final cleaned dataset
print("Final output shape:", df_output.shape)

# Save the cleaned dataframe to a Parquet file for efficient storage
df_output.to_parquet(os.path.join(data_path, "yellow_tripdata_cleand.parquet"), index=False)
# Confirm the save operation and display the shape again
print("Cleaned output saved. Shape:", df_output.shape)

Final output shape: (40356517, 13)
Cleaned output saved. Shape: (40356517, 13)


In [19]:
# Assistant
# Create a random sample of 1000 rows from the cleaned dataset
# Using random_state=42 ensures reproducibility of the sampling
df_sample = df_output.sample(n=1000, random_state=42)

# Write the sample to a CSV file in the specified data path
# index=False prevents writing row indices to the CSV file
df_sample.to_csv(os.path.join(data_path, "yellow_tripdata_cleand_sample_1000.csv"), index=False)

# Print confirmation message and display the dimensions of the sample dataframe
print("Sample saved. Shape:", df_sample.shape)

Sample saved. Shape: (1000, 13)


In [20]:
# Assistant
import pandas as pd
import glob
import os

# Define the path to the data directory
data_path = "Data"

try:
    # Check if df_output already exists in memory
    df_output
    print("Process further from memory.")
except NameError:
    # If df_output doesn't exist, load it from the parquet file
    df_output = pd.read_parquet(os.path.join(data_path, "yellow_tripdata_cleaned.parquet"))
    print("Process further from disk.")

Process further from memory.


In [ ]:
# Assistant
# Create random sample for Tableau
import pandas as pd

# Sample 5% of the original dataframe with a fixed random seed for reproducibility
sampled_df = (df_output.sample(frac=0.05, random_state=42))

# Reset index (nice for Tableau extracts)
sampled_df = sampled_df.reset_index(drop=True)

# Print statistics about the sampling
print("Sampled rows:", len(sampled_df))  # Display the number of rows in the sample
print("Sampling fraction:", len(sampled_df) / len(df_output))  # Calculate and display the actual sampling ratio

In [ ]:
# Assistant
# Define the output file path for the cleaned dataset
output_path = r"Data\tableau_trips_cleaned.csv"

# Save the sampled dataframe to a CSV file without including the index
sampled_df.to_csv(
    output_path,
    index=False
)

# Print confirmation message showing where the file was saved
print(f"Saved Tableau dataset to: {output_path}")

In [ ]:
sampled_df.head()